In [ ]:
from pyspark.sql import SparkSession
''
spark = (SparkSession
         .builder
         .appName('Analyzing the vocabulary of Pride and Prejudice')
         .getOrCreate())

: 

In [ ]:
import requests 
import tempfile

url = 'https://www.gutenberg.org/cache/epub/1342/pg1342.txt'
response = requests.get(url)
text = response.text

with tempfile.NamedTemporaryFile(delete=False, mode='w') as temp_file:
    temp_file.write(text)
    temp_path = temp_file.name

book = spark.read.text(temp_path)

In [ ]:
import pyspark.sql.functions as F

results = (
    book
    .select(F.split(F.col('value'), ' ').alias('line'))
    .select(F.explode(F.col('line')).alias('word'))
    .select(F.lower(F.col('word')).alias('word'))
    .select(F.regexp_extract(F.col('word'), '[a-z]+',0).alias('word'))
    .filter(F.col('word') != '')
    .groupBy('word')
    .count()
)

In [ ]:
results.orderBy(F.col('count').desc()).show()
results.write.csv('data/vocabulary_count.csv')